In [1]:
# some magic so that the notebook will reload external python modules;
# see https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
import json
import random
import os
import math
import numpy as np
import copy

## For Phrases Dataset

In [3]:
label_filepath = "/ssd_scratch/cvit/vanshg/phrases_dataset/akshat_phrases/labels.txt"
label_dir = os.path.dirname(label_filepath)
print(label_dir)

/ssd_scratch/cvit/vanshg/phrases_dataset/akshat_phrases


In [ ]:
label_lines = open(label_filepath, 'r').readlines()

print(len(label_lines))

In [6]:
label_lines = open(label_filepath, 'r').readlines()

# rng = random.Random(42)
# rng.shuffle(label_lines)
print(label_lines[0])

akshat_phrases/processed_videos/1720866143961_0_6692555fbc1718e22960f6ea.mp4 I NEED HELP GETTING UP PLEASE



In [7]:
label_lines = open(label_filepath, 'r').readlines()
rng = random.Random(42)

N = len(label_lines)
test_size = 0.3
train_label_filepath = os.path.join(label_dir, "train_labels.txt")
test_label_filepath = os.path.join(label_dir, "test_labels.txt")

num_train = 200
num_test = 50

# rng.shuffle(label_lines)

# Creating the Train label file
with open(train_label_filepath, 'w') as file:
    for i in range(num_train):
        print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])

# Creating the Test label file
with open(test_label_filepath, 'w') as file:
    for i in range(num_train, N):
        print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])

0, akshat_phrases/processed_videos/1720866143961_0_6692555fbc1718e22960f6ea.mp4 I NEED HELP GETTING UP PLEASE
1, akshat_phrases/processed_videos/1720866178398_1_66925582bc1718e22960f6f9.mp4 CAN YOU BRING ME SOME WATER
2, akshat_phrases/processed_videos/1720866196056_2_66925594bc1718e22960f70c.mp4 HOW ARE YOU FEELING TODAY
3, akshat_phrases/processed_videos/1720866213073_3_669255a5bc1718e22960f723.mp4 WHAT WOULD YOU LIKE FOR DINNER
4, akshat_phrases/processed_videos/1720866246963_4_669255c6bc1718e22960f73e.mp4 COULD YOU CALL THE DOCTOR FOR ME
5, akshat_phrases/processed_videos/1720866263447_5_669255d7bc1718e22960f75d.mp4 I'M HAVING TROUBLE BREATHING
6, akshat_phrases/processed_videos/1720866272634_6_669255e0bc1718e22960f780.mp4 CAN YOU HELP ME TO THE BATHROOM
7, akshat_phrases/processed_videos/1720866282294_7_669255eabc1718e22960f7a7.mp4 MY ARM HURTS A LOT TODAY
8, akshat_phrases/processed_videos/1720866298514_8_669255fabc1718e22960f7d2.mp4 LET'S TALK ABOUT YOUR DAY AT SCHOOL
9, akshat_

## For Deaf Youtube Dataset

In [3]:
data_dir = "/ssd_scratch/cvit/vanshg/datasets/deaf-youtube"
speaker_name = "benny-large"
speaker_dir = os.path.join(data_dir, f"{speaker_name}")

label_filepath = os.path.join(speaker_dir, f"labels.txt")
label_dir = os.path.dirname(label_filepath)
print(label_dir)

/ssd_scratch/cvit/vanshg/datasets/deaf-youtube/benny-large


In [4]:
label_lines = open(label_filepath, 'r').readlines()

# Random shuffle of the labels
rng = random.Random(42)
rng.shuffle(label_lines)
print(label_lines[0])

benny-large/processed_videos/W1hE8AL2Jx8_12_70.mp4 WHEN I WAS MOVED TO SECONDARY SCHOOL AT YEAR 7 I WAS PRETTY NERVOUS BECAUSE IT WAS A BIG SCHOOL



### Creating Train Val Test sets

In [8]:
label_lines = open(label_filepath, 'r').readlines()
rng = random.Random(42)

N = len(label_lines)
test_size = 0.15
val_size = 0.10
train_size = 1 - test_size - val_size
train_label_filepath = os.path.join(label_dir, "train_labels.txt")
val_label_filepath = os.path.join(label_dir, "val_labels.txt")
test_label_filepath = os.path.join(label_dir, "test_labels.txt")

num_test = math.ceil(test_size * N)
num_val = math.ceil(val_size * N)
num_train = N - num_test - num_val
print(f"{num_test = } | {num_val = } | {num_train = }")

rng.shuffle(label_lines)

# Creating the Test label file
with open(test_label_filepath, 'w') as file:
    for i in range(num_test):
        # print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])

# Creating the val label file
with open(val_label_filepath, 'w') as file:
    for i in range(num_test, num_test + num_val):
        # print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])

# Creating the Train label file
with open(train_label_filepath, 'w') as file:
    for i in range(num_test + num_val, N):
        # print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])


num_test = 400 | num_val = 267 | num_train = 1997


### Creating Label files for each video

In [16]:
label_lines = open(label_filepath, 'r').readlines()
print(len(label_lines))

video_name = ''
video_label_lines = []

for label_idx, label_line in enumerate(label_lines):
    video_filename = label_line.split(' ')[0]
    video_fname = os.path.basename(video_filename).split('_')[0]

    if video_name == '':
        video_name = video_fname

    if video_fname != video_name:
        video_label_file = os.path.join(label_dir, f"{video_name}.txt")
        video_lines = ''.join(video_label_lines)
        with open(video_label_file, 'w') as file:
            file.write(video_lines)
            print(f"Wrote {len(video_label_lines)} labels for video {video_name} to {video_label_file}")

        video_label_lines = []
        video_name = video_fname
    
    video_label_lines.append(label_line)

# Writing the label file for the last remaining video
video_label_file = os.path.join(label_dir, f"{video_name}.txt")
video_lines = ''.join(video_label_lines)
with open(video_label_file, 'w') as file:
    file.write(video_lines)
    print(f"Wrote labels for video {video_name} to {video_label_file}")

398
Wrote 92 labels for video JaB9BT09nSE to /ssd_scratch/cvit/vanshg/datasets/deaf-youtube/benny/JaB9BT09nSE.txt
Wrote 107 labels for video XlEO7pWAc84 to /ssd_scratch/cvit/vanshg/datasets/deaf-youtube/benny/XlEO7pWAc84.txt
Wrote 115 labels for video dyb71EMatR0 to /ssd_scratch/cvit/vanshg/datasets/deaf-youtube/benny/dyb71EMatR0.txt
Wrote 34 labels for video jSk5T2aGqpM to /ssd_scratch/cvit/vanshg/datasets/deaf-youtube/benny/jSk5T2aGqpM.txt
Wrote labels for video laRIaxM82dw to /ssd_scratch/cvit/vanshg/datasets/deaf-youtube/benny/laRIaxM82dw.txt


# For Lip2Wav Dataset

In [5]:
data_dir = "/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset"
speaker_name = "chem"
speaker_dir = os.path.join(data_dir, f"{speaker_name}")

label_filepath = os.path.join(speaker_dir, f"all_labels.txt")
label_dir = os.path.dirname(label_filepath)
print(label_dir)

/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/chem


In [6]:
label_lines = open(label_filepath, 'r').readlines()

# Random shuffle of the labels
rng = random.Random(42)
rng.shuffle(label_lines)
print(f"{len(label_lines) = }")
print(label_lines[0])

len(label_lines) = 9314
chem/processed_videos/DnySy87He-A_1_20_18.mp4 SO I CAN LOOK AT THIS AND I SAY WELL BELOW THE FIRST PKA THE ACID FORM PREDOMINATES



In [7]:
label_lines = open(label_filepath, 'r').readlines()
rng = random.Random(42)

N = len(label_lines)
test_size = 0.2
val_size = 0.1
train_size = 1 - test_size - val_size
train_label_filepath = os.path.join(label_dir, "train_labels.txt")
val_label_filepath = os.path.join(label_dir, "val_labels.txt")
test_label_filepath = os.path.join(label_dir, "test_labels.txt")

num_test = math.ceil(test_size * N)
num_val = math.ceil(val_size * N)
num_train = N - num_test - num_val
print(f"{num_test = } | {num_val = } | {num_train = }")

rng.shuffle(label_lines)

# Creating the Test label file
with open(test_label_filepath, 'w') as file:
    for i in range(num_test):
        # print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])

# Creating the val label file
with open(val_label_filepath, 'w') as file:
    for i in range(num_test, num_test + num_val):
        # print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])

# Creating the Train label file
with open(train_label_filepath, 'w') as file:
    for i in range(num_test + num_val, N):
        # print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])

num_test = 735 | num_val = 368 | num_train = 2568


### Creating Same Train-Val-Test splits as Rudrabha

In [8]:
data_dir = "/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset"
speaker = "chem"
speaker_dir = os.path.join(data_dir, f"{speaker}")
video_dir = os.path.join(speaker_dir, "processed_videos")
label_file = os.path.join(speaker_dir, f"all_labels.txt")
label_lines = open(label_file, 'r').readlines()
print(f"{len(label_lines) = }")

# Getting all different video ids
train_video_ids = open(os.path.join(speaker_dir, "train.txt")).read().split()
val_video_ids = open(os.path.join(speaker_dir, "val.txt")).read().split()
test_video_ids = open(os.path.join(speaker_dir, "test.txt")).read().split()

len(label_lines) = 9314


In [9]:
train_label_file = os.path.join(speaker_dir, "train_orig_labels.txt")
val_label_file = os.path.join(speaker_dir, "val_orig_labels.txt")
test_label_file = os.path.join(speaker_dir, "test_orig_labels.txt")

train_label_lines = []
val_label_lines = []
test_label_lines = []

for idx, line in enumerate(label_lines):
    rel_path, gt_text = line.split()[0], " ".join(line.split()[1:])
    video_id_list = os.path.basename(rel_path).split('_')
    video_id = '_'.join(video_id_list[:-3])

    if video_id in train_video_ids:
        train_label_lines.append(copy.deepcopy(line))
    elif video_id in val_video_ids:
        val_label_lines.append(copy.deepcopy(line))
    elif video_id in test_video_ids:
        test_label_lines.append(copy.deepcopy(line))
    else:
        raise KeyError(name=f"{video_id} doesn't belong either train, val or test splits")

# Writing the Train Label Lines
with open(train_label_file, 'w') as file:
    print(f"{len(train_label_lines) = }")
    for line in train_label_lines:
        file.write(line)
    
# Writing the Val Label Lines
with open(val_label_file, 'w') as file:
    print(f"{len(val_label_lines) = }")
    for line in val_label_lines:
        file.write(line)

# Writing the Test Label Lines
with open(test_label_file, 'w') as file:
    print(f"{len(test_label_lines) = }")
    for line in test_label_lines:
        file.write(line)

len(train_label_lines) = 8403
len(val_label_lines) = 473
len(test_label_lines) = 438


### Creating Smaller train files

In [4]:
all_train_path = "/ssd_scratch/cvit/vanshg/Lip2Wav/Dataset/eh/train_labels.txt"
label_dir = os.path.dirname(all_train_path)
label_lines = open(all_train_path, 'r').readlines()
rng = random.Random(42)

N = len(label_lines)
num_train = 1000
train_label_filepath = os.path.join(label_dir, f"train_labels{num_train}.txt")
print(f"Total Train: {N} | {num_train = }")

rng.shuffle(label_lines)

# Creating the Smaller Train label file
with open(train_label_filepath, 'w') as file:
    for i in range(num_train):
        print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])

Total Train: 2568 | num_train = 1000
0, eh/processed_videos/QhTuW5M2t6c_3_28.mp4 NOW THE THING IS THAT IN A NORMAL CHANNEL IF YOU SEND AN ENCRYPTED MESSAGE AND IF I
1, eh/processed_videos/-tfNAI_PwwQ_1_68.mp4 IT CAN TAKE A LOT OF PACKET EXCHANGES TO UPDATE ALL THE ROUTING TABLE TO REFLECT THE CORRECT STATUS OK
2, eh/processed_videos/QhTuW5M2t6c_3_133.mp4 GRAY SCALE IMAGES WHERE YOU ARE NOT TALKING OF COLORS BUT BLACK AND WHITE AND SHADES OF BLACK GRAY THESE IMAGES WORK AND AS I TOLD YOU THESE ARE VULNERABILITY IMAGE MANIPULATIONS
3, eh/processed_videos/-tfNAI_PwwQ_1_8.mp4  MULTIPLE ROUTERS THEY COMMUNICATE USING THIS PROTOCOLS FOR UPDATING THEIR ROUTING TABLES
4, eh/processed_videos/HRw1ORc7Hz0_4_20.mp4  DEPENDING ON THE COMPLEXITY FIRST WE CAN TALK ABOUT BLACK BOX TESTING WHERE YOU DO NOT KNOW OR WE DO NOT KNOW ANYTHING
5, eh/processed_videos/ifiFTkyrUIA_2_9.mp4 SO THERE ARE OTHER ALGORITHMS WHICH HAVE BEEN PROPOSED THIS IDEA IS ONE OF THE ALGORITHMS WHICH HAVE BEEN EXPLORED HERE ALSO